# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('C:/Users/ada10/Desktop/EDAN20/ilppp-master/ilppp-master/programs/corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text):
    words = re.findall('\p{L}+', text.lower())
    return words

In [6]:
words = tokenize(corpus)
words[:10]

['selma',
 'lagerlöf',
 'nils',
 'holgerssons',
 'underbara',
 'resa',
 'genom',
 'sverige',
 'första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
len(set(re.findall(r'\p{L}+', corpus, re.UNICODE)))

44256

Lowercased text

In [8]:
len(set(re.findall(r'\p{L}+', corpus.lower(), re.UNICODE)))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
nonletter=r'[^\p{L}.;:?!]'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
def clean(text):
    text=re.sub(nonletter,' ', text)
    return text


In [11]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort!, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [12]:
test_para = clean(test_para)
print(test_para)

En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa. Hon var nog sina tre alnar lång  hon hade ett stort!  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.


#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [13]:
sentence_boundaries=r'[!,;:?.]\p{Z}+(\p{Lu})'


##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [14]:
sentence_markup=r' </s>\n<s> \1'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [15]:
text=re.sub(sentence_boundaries, sentence_markup, test_para )


In [16]:
print(text)

En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort!  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [17]:
text=re.sub(r'(\p{Lu})',r'<s> \1', text, 1)
text=re.sub(r'([.?!])',r'</s>', text)


In [18]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort</s>  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden</s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [19]:
text=re.sub(r'[\p{Z}]{2,}', ' ', text)

In [20]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort</s> grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden</s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [21]:
def segment_sentences(text):
    text=clean(text)
    text=re.sub(sentence_boundaries, sentence_markup, text)
    text=re.sub(r'(\p{Lu})',r'<s> \1', text, 1)
    text=re.sub(r'[.!?]',r' </s>', text)
    text=re.sub(r'[\p{Z}]{2,}', ' ', text)
    text=text.lower()
    return text
    

In [22]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort </s> grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor; som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [23]:
corpus=segment_sentences(corpus)

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [24]:
print(corpus[-557:])

s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s> 


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [25]:
corpus= re.sub(r'\n+', '\n', corpus)
words = corpus.split()

The five last lines of the corpus should like this:

In [26]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [27]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [28]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 269),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 60928),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11622),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [29]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [30]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 19),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [31]:
def unigram_lm(frequency, wordlings):
    unigram_probabilities = {}
    total=len(words)
    for word in frequency.keys():
        unigram_probabilities[word] = frequency[word]/total
    probability = 1
    entropy = 0
    print ("Unigram model")
    print ("="*54)
    print ("wi    ", "C(wi)    ", "#words    ", "P(wi)")
    print ("="*54)

    for word in wordlings:
        print(
        word,'   ',
        str(frequency[word]),'   ',
        str(total), '   ',
        str(unigram_probabilities[word])
      )
        probability *= unigram_probabilities[word]
        entropy += math.log2(unigram_probabilities[word])

    print ("="*54)

    print ("Prob. unigrams:", probability)
    print ("Geometric mean prob.:", math.pow(probability, 1/len(wordlings)))
    entropy = -1/len(wordlings)*entropy
    print ("Entropy rate:", entropy)
    print ("Perplexity:", math.pow(2,entropy))
    print ()
    Perplexity=math.pow(2,entropy)
    return Perplexity

In [32]:
sentence = 'det var en gång en katt som hette nils </s>'
sentence1 ='hej jag heter selma lagerlöf </s>'
sentence2 = 'i en katt bor ett hus som är rött </s>'
sentence3 = 'i en skog fanns en björn </s>'
sentence4 = 'denna uppgift tog en evighet att göra </s>'
sentence5 = 'jag gillar denna kurs </s>'
sent_words = sentence.split()
sent_words1 = sentence1.split()
sent_words2 = sentence2.split()
sent_words3 = sentence3.split()
sent_words4 = sentence4.split()
sent_words5 = sentence5.split()

print(sent_words, sent_words1, sent_words2, sent_words3, sent_words4, sent_words5)

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>'] ['hej', 'jag', 'heter', 'selma', 'lagerlöf', '</s>'] ['i', 'en', 'katt', 'bor', 'ett', 'hus', 'som', 'är', 'rött', '</s>'] ['i', 'en', 'skog', 'fanns', 'en', 'björn', '</s>'] ['denna', 'uppgift', 'tog', 'en', 'evighet', 'att', 'göra', '</s>'] ['jag', 'gillar', 'denna', 'kurs', '</s>']


In [33]:
perplexity_unigrams = unigram_lm(frequency, sent_words)
perplexity_unigrams1 = unigram_lm(frequency, sent_words1)
perplexity_unigrams2 = unigram_lm(frequency, sent_words2)
perplexity_unigrams3 = unigram_lm(frequency, sent_words3)
perplexity_unigrams4 = unigram_lm(frequency, sent_words4)
perplexity_unigrams5 = unigram_lm(frequency, sent_words5)

Unigram model
wi     C(wi)     #words     P(wi)
det     21107     1043448     0.020228128282386855
var     12089     1043448     0.011585627649868513
en     13513     1043448     0.01295033389301623
gång     1331     1043448     0.0012755786584477617
en     13513     1043448     0.01295033389301623
katt     16     1043448     1.5333778012895706e-05
som     16288     1043448     0.015609786017127831
hette     97     1043448     9.296102920318023e-05
nils     87     1043448     8.33774179451204e-05
</s>     60928     1043448     0.058391026673106854
Prob. unigrams: 5.431077993705472e-27
Geometric mean prob.: 0.002363135388825589
Entropy rate: 8.725081998053719
Perplexity: 423.1666136136921

Unigram model
wi     C(wi)     #words     P(wi)
hej     3     1043448     2.875083377417945e-06
jag     9510     1043448     0.009114014306414887
heter     78     1043448     7.475216781286657e-05
selma     52     1043448     4.983477854191105e-05
lagerlöf     269     1043448     0.0002577991428418091

In [34]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams



423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [35]:
def bigram_lm(frequency, frequency_bigrams, wordlings):
    total=len(words)
    bigram_probabilities={}
    unigram_probabilities={}
    wordlings.insert(0, '<s>')
    
    for word in frequency.keys():
        unigram_probabilities[word] = frequency[word]/total
    
    for bigram in frequency_bigrams.keys():
        bigram_probabilities[bigram] = frequency_bigrams[bigram]/frequency[bigram[0]]
        
    bigrams = [tuple(wordlings[inx:inx+2]) for inx in range(len(wordlings)-1)]

    probability = 1
    entropy = 0

    print ("Bigram model")
    print ("="*54)
    print ("wi wi+1", "Ci,i+1", "C(i)", "P(wi+1|wi)", "backoff")
    print ("="*54)

    for bigram in bigrams:
        bigram_str = str(bigram[0]) + " " + str(bigram[1])  
        prob = 0
        freq = 0
        backoff = ""
        if bigram in bigram_probabilities:
            freq = frequency_bigrams[bigram] 
            prob = bigram_probabilities[bigram]
        else:
            backoff = "backoff:" + bigram[1]
            prob = unigram_probabilities[bigram[1]]

        print(
            bigram_str,
            str(freq),
            str(frequency[bigram[0]]),
            str(prob),
            backoff
        )

        probability *= prob
        entropy += math.log2(prob)

    print("="*54)

    print("Prob. bigrams:", str(probability))
    print("Geometric mean prob.:", str(math.pow(probability, 1/len(wordlings))))
    entropy = -1/len(bigrams)*entropy
    print("Entropy rate:", str(entropy))
    print("Perplexity:", math.pow(2, entropy))
    Perplexity = math.pow(2, entropy)
    return Perplexity

In [36]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)
perplexity_bigrams1 = bigram_lm(frequency, frequency_bigrams, sent_words1)
perplexity_bigrams2 = bigram_lm(frequency, frequency_bigrams, sent_words2)
perplexity_bigrams3 = bigram_lm(frequency, frequency_bigrams, sent_words3)
perplexity_bigrams4 = bigram_lm(frequency, frequency_bigrams, sent_words4)
perplexity_bigrams5 = bigram_lm(frequency, frequency_bigrams, sent_words5)

Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi) backoff
<s> det 5672 59047 0.09605907158704083 
det var 3838 21107 0.18183541005353673 
var en 712 12089 0.05889651749524361 
en gång 705 13513 0.05217198253533634 
gång en 20 1331 0.015026296018031555 
en katt 6 13513 0.0004440168726411604 
katt som 2 16 0.125 
som hette 45 16288 0.002762770137524558 
hette nils 0 97 8.33774179451204e-05 backoff:nils
nils </s> 2 87 0.022988505747126436 
Prob. bigrams: 2.370331925761903e-19
Geometric mean prob.: 0.0202675304079874
Entropy rate: 6.187154470402314
Perplexity: 72.86501915348336
Bigram model
wi wi+1 Ci,i+1 C(i) P(wi+1|wi) backoff
<s> hej 1 59047 1.6935661422256845e-05 
hej jag 0 3 0.009114014306414887 backoff:jag
jag heter 4 9510 0.0004206098843322818 
heter selma 0 78 4.983477854191105e-05 backoff:selma
selma lagerlöf 11 52 0.21153846153846154 
lagerlöf </s> 32 269 0.11895910780669144 
Prob. bigrams: 8.141621634395843e-17
Geometric mean prob.: 0.005029561590907225
Entropy rate: 8.907910239403314

In [37]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [38]:
def trigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i+2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [39]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

329

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [40]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [41]:
cand_nbr = 5

In [42]:
candidates=[]
topcandidates=[]
test=set()
current_word_predictions_1=[]

for key in frequency.keys():
    if key.startswith("de"):
        topcandidates.append((key,frequency[key])) 
        test.add(key)

total_words=len(frequency)

def get_prob_unigram(word):
    if word not in frequency:
        return 0
    return frequency[word]/total_words

def get_prob_bigram(word):
    if word not in frequency_bigrams:
        return 0
    return frequency_bigrams[word]/frequency[word[0]]

def get_prob_trigram(words):
    if words not in frequency_trigrams:
        return 0
    return frequency_trigrams[words] / frequency_bigrams[words[:2]]

for word in test:
    p1=get_prob_unigram((word))
    p2=get_prob_bigram((words[-1], word))
    p=p1+p2
    candidates.append((word, p))

candidates.sort(key=lambda x: x[1], reverse=True)
for ans in candidates[:5]:
    current_word_predictions_1.append(ans[0])

        



In [43]:
current_word_predictions_1

['det', 'de', 'den', 'dem', 'detta']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [44]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [45]:
tokens=current_text.split()

In [46]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [47]:
vocab=set()
candidates=[]
next_word_predictions=[]
for key in frequency.keys():
    if key not in vocab:
        vocab.add(key)

for word in vocab:
    p1 = get_prob_unigram((word))
    p2 = get_prob_bigram((tokens[-1], word))
    p3 = get_prob_trigram((tokens[-2], tokens[-1], word)) if len(tokens) >= 3 else 0
    p=p1*0.01+p2*0.4+p3*0.5
    candidates.append((word, p))

candidates.sort(key=lambda x: x[1], reverse=True)
for ans in candidates[:5]:
    next_word_predictions.append(ans[0])

In [48]:
next_word_predictions

['stor', 'liten', 'gång', 'sådan', 'av']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [49]:
current_text = "Det var en g".lower()

In [50]:
topcandidates=[]
candidates=[]
test=set()
current_word_predictions_2=[]

for key in frequency.keys():
    if key.startswith("g"):
        topcandidates.append((key,frequency[key])) 
        test.add(key)
        
for word in test:
    if word!='</s>' or word!='<s>':
        p1 = get_prob_unigram((word))
        p2 = get_prob_bigram((tokens[-1], word))
        p3 = get_prob_trigram((tokens[-2], tokens[-1], word)) if len(tokens) >= 3 else 0
        p=p1*0.01+p2*0.4+p3*0.5
        candidates.append((word, p))

candidates.sort(key=lambda x: x[1], reverse=True)
for ans in candidates[:5]:
    current_word_predictions_2.append(ans[0])



In [51]:
current_word_predictions_2

['gång', 'gammal', 'god', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [52]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'dem', 'detta'],
 ['stor', 'liten', 'gång', 'sådan', 'av'],
 ['gång', 'gammal', 'god', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [53]:
STIL_ID = ["ada10oco", "student2"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [54]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "dem", "detta"], "next_word_predictions": ["stor", "liten", "g\\u00e5ng", "s\\u00e5dan", "av"], "current_word_predictions_2": ["g\\u00e5ng", "gammal", "god", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [55]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [56]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [57]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'incorrect',
 'signature': None,
 'submission_id': 'c9d4c4bf-c4c1-4540-a3d5-c5c9409d343b'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to include the unigram and bigram tables for five sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 24, 2021.